In [125]:
import numpy as np
import pandas as pd
import os
import geoplot as gplt
import geopandas as geopd
import geoplot.crs as gcrs
from numpy.ma.extras import unique

In [135]:
town_cords = pd.read_csv("/Users/jwilliamson/git/plasma_particle_simulator/Apple-mapping/codes/town cords.csv", index_col=False)
town_cords.head(10)

,Location,Latitude,Longitude
0,Anson,44.80,-69.90
1,Ashland,46.62,-68.40
2,Auburn city,44.09,-70.25
3,Augusta city,44.33,-69.73
4,Bangor city,44.83,-68.79
5,Bar Harbor,44.38,-68.21
6,Bath city,43.93,-69.83
7,Belfast city,44.43,-69.03
8,Berwick,43.27,-70.86
9,Biddeford city,43.44,-70.40


In [136]:
county_cords = pd.read_csv("/Users/jwilliamson/git/plasma_particle_simulator/Apple-mapping/cord data/CountyCords.csv", index_col=False)
county_cords

,county,lat,long
0,Androscoggin,-70.198,44.187
1,Aroostook,-68.412,46.574
2,Cumberland,-70.268,43.874
3,Franklin,-70.449,44.980
4,Hancock,-68.402,44.614
5,Kennebec,-69.753,44.415
6,Knox,-69.135,44.101
7,Lincoln,-69.518,44.078
8,Oxford,-70.734,44.426
9,Penobscot,-68.575,45.354


In [ ]:
def clean_country(value):
    if isinstance(value, str):
        return value.lower().replace("country", "").strip()

In [127]:
def clean_city(value):
    if isinstance(value, str):
        return value.lower().replace("city", "").strip()
    return value

In [150]:
town_cords_clean = town_cords
town_cords_clean['Location'] = town_cords["Location"].apply(clean_city)
town_cords_clean

,Location,Latitude,Longitude
0,anson,44.80,-69.90
1,ashland,46.62,-68.40
2,auburn,44.09,-70.25
3,augusta,44.33,-69.73
4,bangor,44.83,-68.79
...,...,...,...
125,winthrop,44.31,-69.96
126,wiscasset,44.01,-69.67
127,woodland,45.16,-67.41
128,yarmouth,43.80,-70.19


In [129]:
source_df = pd.read_csv("/Users/jwilliamson/git/plasma_particle_simulator/Apple-mapping/csv/reformated_sources.csv", index_col=False, keep_default_na=False)
source_df

,County,Fair,Year,Event,Location,Source,Page,Notes
0,piscataquis,piscataquis central fair,1912.0,piscataquis central fair,foxcroft,piscataquis observer,3,premiums
1,piscataquis,piscataquis central fair,1894.0,piscataquis central fair,foxcroft,piscataquis observer,2,premiums
2,piscataquis,piscataquis central fair,1892.0,piscataquis central fair,foxcroft,piscataquis observer,3,premiums
3,piscataquis,piscataquis central fair,1884.0,piscataquis central fair,foxcroft,piscataquis observer,3,premiums
4,piscataquis,piscataquis central fair,1887.0,piscataquis central fair,,piscataquis observer,2,description
...,...,...,...,...,...,...,...,...
1276,waldo,monroe,1915.0,north waldo and penobscot fair,monroe,republican journal,,premiums
1277,waldo,monroe,1911.0,north waldo and penobscot fair,monroe,republican journal,3,premiums
1278,waldo,monroe,1888.0,north waldo and penobscot fair,monroe,maine farmer,2,description
1279,waldo,monroe,1891.0,north waldo and penobscot fair,monroe,maine farmer,2,premiums


In [130]:
def clean_location(value):
    if isinstance(value, str):
        return (value.lower()
                 .replace("foxcroft", "dover-foxcroft")
                 .strip())
    return value

In [131]:
clean_source_df = source_df
clean_source_df['Location'] = clean_source_df["Location"].apply(clean_location)
mask = clean_source_df["Location"] == ""
clean_source_df["Missing Location"] = mask
clean_source_df


,County,Fair,Year,Event,Location,Source,Page,Notes,Missing Location
0,piscataquis,piscataquis central fair,1912.0,piscataquis central fair,dover-foxcroft,piscataquis observer,3,premiums,False
1,piscataquis,piscataquis central fair,1894.0,piscataquis central fair,dover-foxcroft,piscataquis observer,2,premiums,False
2,piscataquis,piscataquis central fair,1892.0,piscataquis central fair,dover-foxcroft,piscataquis observer,3,premiums,False
3,piscataquis,piscataquis central fair,1884.0,piscataquis central fair,dover-foxcroft,piscataquis observer,3,premiums,False
4,piscataquis,piscataquis central fair,1887.0,piscataquis central fair,,piscataquis observer,2,description,True
...,...,...,...,...,...,...,...,...,...
1276,waldo,monroe,1915.0,north waldo and penobscot fair,monroe,republican journal,,premiums,False
1277,waldo,monroe,1911.0,north waldo and penobscot fair,monroe,republican journal,3,premiums,False
1278,waldo,monroe,1888.0,north waldo and penobscot fair,monroe,maine farmer,2,description,False
1279,waldo,monroe,1891.0,north waldo and penobscot fair,monroe,maine farmer,2,premiums,False


In [132]:
nonull_loc_source_df = clean_source_df[~mask]
nonull_loc_source_df

,County,Fair,Year,Event,Location,Source,Page,Notes,Missing Location
0,piscataquis,piscataquis central fair,1912.0,piscataquis central fair,dover-foxcroft,piscataquis observer,3,premiums,False
1,piscataquis,piscataquis central fair,1894.0,piscataquis central fair,dover-foxcroft,piscataquis observer,2,premiums,False
2,piscataquis,piscataquis central fair,1892.0,piscataquis central fair,dover-foxcroft,piscataquis observer,3,premiums,False
3,piscataquis,piscataquis central fair,1884.0,piscataquis central fair,dover-foxcroft,piscataquis observer,3,premiums,False
5,piscataquis,piscataquis central fair,1882.0,piscataquis central fair,dover-foxcroft,piscataquis observer,2,premiums,False
...,...,...,...,...,...,...,...,...,...
1275,waldo,monroe,1914.0,north waldo and penobscot fair,monroe,republican journal,,premiums,False
1276,waldo,monroe,1915.0,north waldo and penobscot fair,monroe,republican journal,,premiums,False
1277,waldo,monroe,1911.0,north waldo and penobscot fair,monroe,republican journal,3,premiums,False
1278,waldo,monroe,1888.0,north waldo and penobscot fair,monroe,maine farmer,2,description,False


In [133]:
merge_nonnull = nonull_loc_source_df.merge(town_cords, on=["Location"], how="left")
merge_nonnull

,County,Fair,Year,Event,Location,Source,Page,Notes,Missing Location,Latitude,Longitude
0,piscataquis,piscataquis central fair,1912.0,piscataquis central fair,dover-foxcroft,piscataquis observer,3,premiums,False,45.19,-69.22
1,piscataquis,piscataquis central fair,1894.0,piscataquis central fair,dover-foxcroft,piscataquis observer,2,premiums,False,45.19,-69.22
2,piscataquis,piscataquis central fair,1892.0,piscataquis central fair,dover-foxcroft,piscataquis observer,3,premiums,False,45.19,-69.22
3,piscataquis,piscataquis central fair,1884.0,piscataquis central fair,dover-foxcroft,piscataquis observer,3,premiums,False,45.19,-69.22
4,piscataquis,piscataquis central fair,1882.0,piscataquis central fair,dover-foxcroft,piscataquis observer,2,premiums,False,45.19,-69.22
...,...,...,...,...,...,...,...,...,...,...,...
1179,waldo,monroe,1914.0,north waldo and penobscot fair,monroe,republican journal,,premiums,False,NaN,NaN
1180,waldo,monroe,1915.0,north waldo and penobscot fair,monroe,republican journal,,premiums,False,NaN,NaN
1181,waldo,monroe,1911.0,north waldo and penobscot fair,monroe,republican journal,3,premiums,False,NaN,NaN
1182,waldo,monroe,1888.0,north waldo and penobscot fair,monroe,maine farmer,2,description,False,NaN,NaN


In [149]:
null_loc_source_df = clean_source_df[mask]
null_loc_source_df

,County,Fair,Year,Event,Location,Source,Page,Notes,Missing Location
4,piscataquis,piscataquis central fair,1887.0,piscataquis central fair,,piscataquis observer,2,description,True
12,piscataquis,piscataquis fair,1929.0,piscataquis fair,,piscataquis observer,4,premiums,True
14,piscataquis,piscataquis fair,1914.0,piscataquis fair,,piscataquis observer,8,premiums,True
15,piscataquis,piscataquis fair,1910.0,piscataquis fair,,piscataquis observer,3,premiums,True
16,piscataquis,piscataquis fair,1909.0,piscataquis fair,,piscataquis observer,1,des. & prem.,True
...,...,...,...,...,...,...,...,...,...
1008,kennebec,gardiner,1865.0,kennebec union ag. & hort. soc. fair,,gardiner home journal,2,premiums,True
1011,kennebec,gardiner,1867.0,kennebec union ag. & hort. soc. fair,,gardiner home journal,1,premiums,True
1012,kennebec,gardiner,1868.0,kennebec union ag. & hort. soc. fair,,gardiner home journal,1,premiums,True
1201,waldo,waldo co.,1886.0,,,,,,True


In [164]:

nonull_loc_source_df[["Location"]].drop_duplicates().merge(town_cords_clean ,on="Location", how="left" ).to_csv("that.csv", index=False)

In [156]:
town_cords_clean

,Location,Latitude,Longitude
0,anson,44.8,-69.9
1,ashland,46.62,-68.4
2,auburn,44.09,-70.25
3,augusta,44.33,-69.73
4,bangor,44.83,-68.79
...,...,...,...
125,winthrop,44.31,-69.96
126,wiscasset,44.01,-69.67
127,woodland,45.16,-67.41
128,yarmouth,43.8,-70.19
